# Agrupando letras de músicas de Samba, Rock e Sertanejo

Letras de músicas segundo o vagalume.com.br, coletadas por Anderson Neisse e [disponíveis no kaggle](https://www.kaggle.com/neisse/scrapped-lyrics-from-6-genres/data).

Os dados originais foram tratados para diminuir duplicações e diminuir o tamanho do arquivo. Os resultados (e outros recortes dos dados de letras) estão [nesse repo](https://github.com/nazareno/palavras-nas-letras).

In [2]:
import pandas as pd
import numpy as np
import altair as alt

In [3]:
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re

[nltk_data] Downloading package stopwords to /home/mateus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
rock_lyrics = pd.read_csv('https://raw.githubusercontent.com/nazareno/palavras-nas-letras/master/letras-ptbr-rock-grande.csv')
samba_lyrics = pd.read_csv('https://raw.githubusercontent.com/nazareno/palavras-nas-letras/master/letras-ptbr-samba-grande.csv')
sertanejo_lyrics = pd.read_csv('https://raw.githubusercontent.com/nazareno/palavras-nas-letras/master/letras-ptbr-sertanejo-grande.csv')


,SName,Lyric,Artist,Songs,Popularity,Genre,Genres
7545,Tão Seu,Eu sinto sua falta. Não posso esperar tanto te...,Skank,139,12.3,Rock,Pop/Rock; Rock; Pop; Rock Alternativo; MPB; Ro...
106,Por Que A Gente é Assim?,Mais uma dose?. É claro que eu estou a fim. A ...,Barão Vermelho,160,3.8,Rock,Rock; Pop/Rock; MPB; Romântico; Rock Alternati...
3911,Mundo Negro,Oh oh oh.oh Oh oh oh oh Oh oh oh oh Oh oh oh. ...,O Rappa,97,10.2,Rock,Reggae; Rap; Rock; Pop/Rock; Black Music; Funk...
4535,Faça O Bem,"Pois, eu conheço bem os teus erros. Prefere o ...",PG,111,1.1,Rock,Rock; Gospel/Religioso; Pop/Rock; Funk; Clássi...
5970,Tanto (I Want You),Coveiros gemem tristes ais. E realejos ancestr...,Skank,139,12.3,Rock,Pop/Rock; Rock; Pop; Rock Alternativo; MPB; Ro...
7290,Um Único Olhar,"Então, me coloco a sua frente. vai descobrir m...",Jota Quest,154,13.3,Rock,Pop/Rock; Pop; Rock; Romântico; Funk; Black Mu...
5978,Um Mais Um,Éramos nós (2x). Um mais um. Éramos mais. Que ...,Skank,139,12.3,Rock,Pop/Rock; Rock; Pop; Rock Alternativo; MPB; Ro...
177,Jardins da Babilônia,Suspenderam os jardins da Babilônia. E eu prá ...,Barão Vermelho,160,3.8,Rock,Rock; Pop/Rock; MPB; Romântico; Rock Alternati...
7302,Waiting For You (Party On),Um novo dia nasceu. O tempo passou e o nosso a...,Jota Quest,154,13.3,Rock,Pop/Rock; Pop; Rock; Romântico; Funk; Black Mu...
282,Em Algum Lugar No Tempo,Não guarde mágoa de mim. Também não me esqueça...,Biquini Cavadão,162,3.4,Rock,Rock; Pop/Rock; Romântico; Punk Rock; New Wave...


#### Observações iniciais
Vamos começar realizando algumas observações superficiais de cada Dataframe, a partir dos métodos sample(), info() e value_counts() de artistas.

In [ ]:
samba_lyrics.sample(10)

In [13]:
sertanejo_lyrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8662 entries, 0 to 8661
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SName       8662 non-null   object 
 1   Lyric       8662 non-null   object 
 2   Artist      8662 non-null   object 
 3   Songs       8662 non-null   int64  
 4   Popularity  8662 non-null   float64
 5   Genre       8662 non-null   object 
 6   Genres      8662 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 473.8+ KB


In [14]:
rock_lyrics['Artist'].value_counts()[0:20]

Lulu Santos                462
Engenheiros do Hawaii      402
Jota Quest                 296
Skank                      290
Rita Lee                   282
Erasmo Carlos              236
Cássia Eller               226
Titãs                      209
Charlie Brown Jr           198
Raul Seixas                196
Capital Inicial            195
Biquini Cavadão            184
Os Paralamas do Sucesso    181
Barão Vermelho             164
Velhas Virgens             156
Fresno                     153
Blitz                      150
Pato Fu                    146
Ira!                       142
Rosa de Saron              137
Name: Artist, dtype: int64

#### Pré-processamento
Faremos o pré-processamento dos textos das letras a partir da exclusão de caracteres especiais

In [15]:
stop_words = set(stopwords.words("portuguese"))
print(len(stop_words))

204


In [22]:
rock_lyrics_c = []
samba_lyrics_c = []
sertanejo_lyrics_c = []

lyrics_list = [rock_lyrics, samba_lyrics, sertanejo_lyrics]
cleaned_lyrics_list = [rock_lyrics_c, samba_lyrics_c, sertanejo_lyrics_c]

for i in range(3):
    lyrics_df = lyrics_list[i]
    for j in range(len(lyrics_df.Lyric)):
        lyric = lyrics_df['Lyric'][j]

        # Removendo caracteres especiais
        lyric = re.sub("(\\d|\\W)+|\w*\d\w*"," ", lyric)
        lyric = ' '.join(s for s in lyric.split() if (not any(c.isdigit() for c in s)) and len(s) > 2)
        cleaned_lyrics_list[i].append(lyric)

rock_lyrics_c[5:6]


['Saudade Espero que logo logo vai passar Vontade Hoje quero matar com você Sempre que puder voltarei aqui Sempre que puder quero ver sorrir Felicidade vale tiver alguém pra dividir vou levar pra ver sol pra ver mar Andar meu caminho levo emoção pra acelerar meu coração Não quero estar sozinho vou levar pra ver sol pra ver mar Andar meu caminho levo emoção pra acelerar meu coração levo comigo aprendi usar solidão vivo bem comigo mesmo então Respeito com liberdade Onde estiver seja bem vinda agora você quiser Sempre que puder voltarei aqui Sempre que puder quero ver sorrir Felicidade vale tiver alguém pra dividir vou levar pra ver sol pra ver mar Andar meu caminho levo emoção pra acelerar meu coração Não quero estar sozinho vou levar pra ver sol pra ver mar Andar meu caminho levo emoção pra acelerar meu coração levo comigo vou levar pra ver sol pra ver mar Andar meu caminho levo emoção pra acelerar meu coração Não quero estar sozinho vou levar pra ver sol pra ver mar Andar meu caminho l

#### Criando vetores TF-IDF

In [24]:
#TF-IDF vectorizer
tfv_rock = TfidfVectorizer(
        min_df = 10,
        max_df = 0.5,
        max_features = None,
        stop_words = stop_words, 
        ngram_range = (1,3)
  )

tfv_samba = TfidfVectorizer(
        min_df = 10,
        max_df = 0.5,
        max_features = None,
        stop_words = stop_words, 
        ngram_range = (1,3)
  )

tfv_sertanejo = TfidfVectorizer(
        min_df = 10,
        max_df = 0.5,
        max_features = None,
        stop_words = stop_words, 
        ngram_range = (1,3)
  )



#transform
vec_text_rock = tfv1.fit_transform(rock_lyrics_c)
vec_text_samba = tfv2.fit_transform(samba_lyrics_c)
vec_text_sertanejo = tfv3.fit_transform(sertanejo_lyrics_c)

#returns a list of words.
rock_words = tfv1.get_feature_names()
samba_words = tfv2.get_feature_names()
sertanejo_words = tfv3.get_feature_names()

print(len(rock_words), len(samba_words), len(sertanejo_words))

8525 9228 16418


/home/mateus/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
